## Primer 3. Maximum Coverage

Funkcije maksimalne pokrivenosti skupa (eng. *maximum coverage functions*) imaju cilj da maksimizuju broj atributa razlicitih od 0.
Podaci su predstavljeni u vidu binarne matrice gde 1 znaci da je stavka prisutna u elementu $x \in X$, a 0 da nije.
Ove funkcije su korisne kada je prostor variabli valik, a svaki element skupa sadrzi samo mali podskup njih - sto je cest slucaj kada se analiziraju tekstualni podaci, gde su variable reci.

Funckija maksimalne pokrivenosti je funkcija zasnovana na atributima (eng *feature based function*), gde je konkavna funkcija *minimum*.
Napomena: Sve vrednosti skupa podataka moraju biti binarne da bi apricot selektor radio.

Opsti oblik funkcije pokrivenosti je: <br />
$f(X)=\displaystyle \sum_{d=1}^{D} min(\displaystyle \sum_{i=1}^{N} X_{i,d},1)$ <br />

gde je $f$ funkcija nad podskupom $X$ koji ima $N$ elemenata i $D$ dimenzija.

Naredni primer je primer iz zvanicne dokumentacije. <br />
Optimizujemo *max coverage* funkciju nad tekstualnim podacima, koja racuna broj jedinstvenih reci u tekstu.

In [1]:
from apricot import MaxCoverageSelection
from keras.datasets import reuters
import numpy as np

Dataset *reuters* : https://keras.io/api/datasets/reuters/ <br />
"This is a dataset of 11,228 newswires from Reuters, labeled over 46 topics." <br />
"Each newswire is encoded as a list of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data."

In [2]:
(X_, _), (_, _) = reuters.load_data(num_words=5000)

In [3]:
X_[:1]

array([list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12])],
      dtype=object)

In [4]:
print('X_.shape: ', X_.shape)

X_.shape:  (8982,)


In [5]:
# Generisem binarnu matricu X
X = np.zeros((X_.shape[0], max(map(max, X_))+1))
for i, x in enumerate(X_):
    X[i][x] = 1

In [6]:
max(max(X_))

4992

In [7]:
print('X.shape: ', X.shape)

X.shape:  (8982, 5000)


In [8]:
# Iskoristicu posebnu podrsku za retke matrice
from scipy.sparse import csr_matrix

X_sparse = csr_matrix(X)

In [14]:
model = MaxCoverageSelection(250, optimizer='naive')
X_transformed = model.fit_transform(X_sparse)

Koriscenjem podrske za retke matrice optimizacija se ubrzava skoro 3x.

In [10]:
X_transformed.shape

(250, 5000)

In [11]:
X_transformed

<250x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 46714 stored elements in Compressed Sparse Row format>